In [1]:
import gradio as gr

import numpy as np
from PIL import Image, ImageDraw, ImageFont

import easyocr

import os
import io
import requests
import base64
import cv2

from datetime import datetime
from uuid import uuid4

from utils import outpainting_with_value, MODELS, VAES, SIZES, BASE_NEGATIV_PROMPT, generate_image_txt2img, get_samplers

import glob

import random

In [38]:
positive_prompts = [
    (
        ["white girl in casino", "portrait  girl at a casino", "girl at a casino"], [0.3, 0.1, 0.5], 0.7
    ),
    (
        ["smiling woman looking into the camera", "smiling woman", "smiling"], [0.1, 0.1, 0.3], 0.5
    ),
    ("casino", 0.3),
    ("<lora:casino:0.6>", 0.2),
    ("<lora:add_detail:1>", 0.05),
    ("<lora:more_details:0.8>", 0.05),
    ("<lora:BeautifulDetailedEyes:0.5>", 0.05),
    ("<lora:polyhedron_all_eyes:0.4>", 0.05),
    (
        ["black corset", "red leotard", "jacket", "red striped sweater"], [0.1, 0.1, 0.2, 0.1], 0.4
    ),
    (
        ["pink hair", "short hair", "orange hair"], [0.1, 0.2, 0.4], 0.2
    ),
   (
        ["light pink eyes", "green eyes"], [0.1, 0.3], 0.2
    ),
    (
        ["realistic", "ultra realistic"], [0.3, 0.4], 0.5
    ),
    ("depth of field insanely detailed", 0.1),
    ("detailed face", 0.1),
    ("octane rendered", 0.05),
    ("masterpiece", 0.05),
    ("high quality", 0.05),
    ("beautiful", 0.05),
    ("unreal engine", 0.05),
    ("shiny skin", 0.05),
    ("surrounded by glowing", 0.05),
    ("real life", 0.05),
    ("detailed background", 0.2),
    ("photoshoot", 0.05),
    ("amazing", 0.05),
    ("Nikon D850 film stock photograph Kodak Portra 400 camera f1.6 lens", 0.05),
    ("amazing fine detail", 0.05),
    ("close-up", 0.05),
    ("round cute face", 0.05),
    ("trending on artstation", 0.05),
    (
        ["sitting on roulette table", "a pinup of a girl sitting on roulette"], [0.3, 0.1], 0.1
    ),
    ("white cuffs", 0.05),
    ("bow tie", 0.2),
    ("photo-realistic", 0.1),
    ("best quality", 0.2),
    ("gambling", 0.05),
    ("roulette wheel spins", 0.05),
    ("cinestill 800 tungsten", 0.05),
    ("indoor", 0.05),
    ("looking at viewer", 0.4),
    ("front", 0.05),
    ("ultra-detailed", 0.05),
    ("professional lighting", 0.05),
    ("roulette", 0.05),
    ("slot machine", 0.05),
    ("in the casino", 0.05),
    ("roulette table", 0.05),
    ("leaning forward", 0.1),
    ("indoors", 0.05),
    ("in casino", 0.05),
    ("stack of coin", 0.05),
    ("spread legs", 0.05),
    ("nose blush", 0.05),
    ("looking up", 0.05),
    ("from above", 0.05),
    ("braid", 0.05),
    ("earrings", 0.01),
    ("glossy luster playboy bunny", 0.05),
    ("oversized breast cup", 0.05),
    ("bare legs", 0.05),
    ("Pin-up pose", 0.05),
    ("pin up", 0.05),
    ("pinup pose", 0.05),
    ("sexy slut poses", 0.1),
    ("latex bdsm suit", 0.1),
    ("latex", 0.1),
    ("looking into the camera", 0.1),
    (
        ["8k", "4k", "16k"], [0.2, 0.2, 0.2], 0.4
    )
]

negative_prompts = [
    ("EasyNegative", 0.3),
    ("worst quality", 0.1),
    ("BadDream", 0.5),
    ("UnrealisticDream", 0.5),
    ("bad anatomy", 0.1),
    ("deformed hands", 0.1),
    ("ugly", 0.1),
]

def create_prompt(probabilities=positive_prompts):
    out_prompt = []
    for prob in probabilities:
        if isinstance(prob[0], str):
            if prob[1] > random.random():
                out_prompt.append(prob[0])
        else:
            # вариант со вложенностью
            if prob[2] > random.random():
                out_prompt.append(random.choices(prob[0], weights=prob[1], k=1)[0])
    return ", ".join(out_prompt)

In [39]:
create_prompt()

'casino, realistic, best quality'

In [40]:
RESULT_DIR = r"IMGS_1_11"
RESULTS = os.listdir(RESULT_DIR)
OUT_DIR = r"WITHOUT_TEXT"

In [41]:
model = "deliberate_v3.safetensors [aadddd3d75]"
vae = "vaeFtMse840000Ema_v100.pt"
size = "Стандарт 408×544"
prompt = "casino, blackjack"
sampler = "DPM++ 2M Karras"
steps = 40
cfg_scale = 7
denoising_strength = 0.75
batch_size = 15

In [42]:
VAES

['None',
 'difconsistencyRAWVAE_v10.pt',
 'klF8Anime2_klF8Anime2VAE.pt',
 'vaeFtMse840000Ema_v100.pt',
 'color101VAE_v1.safetensors',
 'toneRangeCompressor_trcvae.safetensors',
 'anythingKlF8Anime2VaeFtMse840000_klF8Anime2.safetensors']

In [43]:
get_samplers()

['DPM++ 2M Karras',
 'DPM++ SDE Karras',
 'DPM++ 2M SDE Exponential',
 'DPM++ 2M SDE Karras',
 'Euler a',
 'Euler',
 'LMS',
 'Heun',
 'DPM2',
 'DPM2 a',
 'DPM++ 2S a',
 'DPM++ 2M',
 'DPM++ SDE',
 'DPM++ 2M SDE',
 'DPM++ 2M SDE Heun',
 'DPM++ 2M SDE Heun Karras',
 'DPM++ 2M SDE Heun Exponential',
 'DPM++ 3M SDE',
 'DPM++ 3M SDE Karras',
 'DPM++ 3M SDE Exponential',
 'DPM fast',
 'DPM adaptive',
 'LMS Karras',
 'DPM2 Karras',
 'DPM2 a Karras',
 'DPM++ 2S a Karras',
 'Restart',
 'DDIM',
 'PLMS',
 'UniPC']

In [44]:
A111_url = "http://127.0.0.1:7860"
def log_img(imgs):
    for img in imgs:
        img.save(os.path.join("log", datetime.now().strftime("%d%m_%H_%M%S") + f"{str(uuid4())[:5]}.png"))

In [45]:
def generate_image_txt2img(
        model, vae_name, prompt, negative_prompt, sampler, steps, cfg_scale, denoising_strength, batch_size=1
):
    payload = {
        "prompt": prompt,
        "negative_prompt": negative_prompt,
        "steps": steps,
        "batch_size": batch_size,
        "override_settings": {
            "sd_model_checkpoint": model,
            "sd_vae": vae_name
        },
        "width": 408,
        "height": 544,
        "sampler_index": sampler,
        "cfg_scale": cfg_scale,
        "denoising_strength": denoising_strength
    }

    response = requests.post(f'{A111_url}/sdapi/v1/txt2img', json=payload)
    imgs = [Image.open(io.BytesIO(base64.b64decode(i))) for i in response.json()["images"]]
    log_img(imgs)
    return imgs

In [ ]:
while True:
    for model in [
        "babes_20.safetensors [d319cb2188]", "westmixV1PhotoRealistic_v10.safetensors [4870599b7e]",
        "neverendingDreamNED_v122BakedVae.safetensors [ecefb796ff]", "dreamshaper_8.safetensors [879db523c3]",
        "epicrealism_naturalSinRC1VAE.safetensors [84d76a0328]", "deliberate_v3.safetensors [aadddd3d75]",
        "westmixV1PhotoRealistic_v10.safetensors [4870599b7e]", "westmixV1PhotoRealistic_v10.safetensors [4870599b7e]"
    ]:
        for vae in ["None", "color101VAE_v1.safetensors", "vaeFtMse840000Ema_v100.pt"]:
            for cfg_scale in [7, 5, 8.5]:
                prompt = create_prompt(positive_prompts)
                neg_prompt = create_prompt(negative_prompts)
                result_image = generate_image_txt2img(
                    model, vae, prompt, neg_prompt, sampler, steps, cfg_scale, denoising_strength, batch_size
                )
                print(prompt)

girl at a casino, casino, <lora:BeautifulDetailedEyes:0.5>, jacket, realistic, detailed face, unreal engine, shiny skin, 16k
<lora:BeautifulDetailedEyes:0.5>, red striped sweater, green eyes, bow tie, best quality, gambling, leaning forward, spread legs, looking up, looking into the camera
white girl in casino, smiling, beautiful, best quality, looking at viewer, 8k
girl at a casino, smiling, orange hair, light pink eyes, ultra realistic, surrounded by glowing, detailed background, bow tie, photo-realistic, best quality, indoor, looking into the camera
girl at a casino, smiling woman, casino, pink hair, sitting on roulette table, best quality, cinestill 800 tungsten, front, glossy luster playboy bunny, 8k
portrait  girl at a casino, smiling woman, casino, jacket, green eyes, realistic, sitting on roulette table
girl at a casino, <lora:casino:0.6>, ultra realistic, close-up, best quality, looking at viewer, professional lighting, pinup pose
girl at a casino, smiling, casino, realistic, 